In [1]:
!pip install datasets==1.0.1
!pip install transformers==3.1.0

     |████████████████████████████████| 1.8 MB 7.5 MB/s 
     |████████████████████████████████| 212 kB 45.1 MB/s 
     |████████████████████████████████| 884 kB 6.9 MB/s 
     |████████████████████████████████| 895 kB 30.8 MB/s 
     |████████████████████████████████| 3.0 MB 23.6 MB/s 
     |████████████████████████████████| 1.2 MB 51.5 MB/s 


In [2]:
import torch
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd

! pip install wiktionaryparser
from wiktionaryparser import WiktionaryParser
parser = WiktionaryParser()
!pip install textblob
from textblob import TextBlob
!pip install pyspellchecker
from spellchecker import SpellChecker


from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
from datasets import load_dataset, load_metric



spell = SpellChecker()


os.environ["TOKENIZERS_PARALLELISM"] = "false"

     |████████████████████████████████| 2.7 MB 6.8 MB/s 


PyTorch version 1.10.0+cu111 available.
TensorFlow version 2.8.0 available.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_train = pd.read_csv('/content/drive/MyDrive/CSCI-544/pnli_train.csv', header=None,  encoding="UTF-8")
df_val   = pd.read_csv('/content/drive/MyDrive/CSCI-544/pnli_dev.csv', header=None, encoding="UTF-8")
df_test = pd.read_csv('/content/drive/MyDrive/CSCI-544/pnli_test_unlabeled.csv', header=None, encoding="UTF-8")
df_train.columns = ["sentence1_unprocessed","sentence2_unprocessed","label"]
df_val.columns = ["sentence1_unprocessed","sentence2_unprocessed","label"]
df_test.columns = ["sentence1_unprocessed","sentence2_unprocessed"]


In [5]:
# Pre Processed Text

# df_train = pd.read_csv('/content/drive/MyDrive/CSCI-544/trainpy.csv', header=0,  encoding="UTF-8")
# df_val   = pd.read_csv('/content/drive/MyDrive/CSCI-544/valpy.csv', header=0, encoding="UTF-8")
# df_test = pd.read_csv('/content/drive/MyDrive/CSCI-544/testpy.csv', header=0, encoding="UTF-8")

In [6]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(5983, 3)
(1055, 3)
(4850, 2)


In [7]:
df_train.head()

,sentence1_unprocessed,sentence2_unprocessed,label
0,Sometimes do exercise.,A person typically desire healthy life.,1
1,Who eats junk foods.,A person typically desire healthy life.,0
2,A person is sick.,A person typically desire healthy life.,1
3,A person is dead.,A person typically desire healthy life.,0
4,A person eats properly and do exercise regularly.,A person typically desire healthy life.,1


In [8]:
# !pip install pyenchant
# import enchant


In [9]:
def remove_stop_words(sentence):
  stop_list = ["a","an","the","typically","is","for","they","their","i", "them","themselves"]
  new_sentence = ""
  for word in sentence.split():
    if word.lower() not in stop_list:
      new_sentence = new_sentence + " " + word
  return new_sentence.strip()

def remove_un(old_sent):
  new_sent =""
  d = enchant.Dict("en_US")
  for word in old_sent.split():
    new_word = word
    if not d.check(word):
      if word.lower().startswith("un") and word.lower() not in ["union","unite","united"]:
        print("Old Word " + word)
        new_word = "not " + word.lower().split('un')[1]
        print(new_word)
    new_sent = new_sent + " " + new_sent
  return new_sent
      

def correct_spelling_pycheck(old_sent):
  old_sent = re.sub(r'[^\w\s]', '', old_sent)
  new_sent =""
  for word in spell.unknown(list(old_sent.split())):
    if word.lower() not in ["tv","php","gps", "aseexuality", "angryman", "fastfood","golddigger","asexuality","covid","bbq", "vcr"] and "un" not in word.lower(): 
      new_word = str(spell.correction(word))
      new_sent = new_sent + " "  + new_word
      if str(word).lower() != new_word.lower():
        print("Old Word: "+ word)
        print("New Word: "+ new_word)
  return new_sent

def data_preprocessing(curr_value):
  new_value = str(curr_value)

  #new_value = str(remove_un(new_value))
  #print(new_value)
  #new_value = str(correct_spelling_pycheck(new_value))
  #print(new_value)

  #print("")
  new_value = str(remove_stop_words(curr_value))
  
  return str(new_value)

def itter_dataset(dataset, column):
  cat_list = []
  for index, row in dataset.iterrows():
      wt = row[column+"_unprocessed"]    
      cat = data_preprocessing(wt)
      cat_list.append(cat)
  #print(cat_list)
  dataset[column+"_3"] = cat_list
  return dataset

In [10]:
df_train["sentence1"] = df_train.apply(lambda row : data_preprocessing(row["sentence1_unprocessed"]), axis=1)
df_val["sentence1"] = df_val.apply(lambda row : data_preprocessing(row["sentence1_unprocessed"]), axis=1)
#df_val = itter_dataset(df_val, "sentence1")
df_test["sentence1"] = df_test.apply(lambda row : data_preprocessing(row["sentence1_unprocessed"]), axis=1)
print(df_val.head())

                       sentence1_unprocessed  \
0          A person is looking for accuracy.   
1       A person does not care for accuracy.   
2       The person double checks their data.   
3  The person speeds through the experiment.   
4                 A person is studying well.   

                          sentence2_unprocessed  label  \
0  A person typically desires accurate results.      1   
1  A person typically desires accurate results.      0   
2  A person typically desires accurate results.      1   
3  A person typically desires accurate results.      0   
4  A person typically desires accurate results.      1   

                           sentence1  
0           person looking accuracy.  
1     person does not care accuracy.  
2         person double checks data.  
3  person speeds through experiment.  
4              person studying well.  


In [11]:
print(df_val.head())

                       sentence1_unprocessed  \
0          A person is looking for accuracy.   
1       A person does not care for accuracy.   
2       The person double checks their data.   
3  The person speeds through the experiment.   
4                 A person is studying well.   

                          sentence2_unprocessed  label  \
0  A person typically desires accurate results.      1   
1  A person typically desires accurate results.      0   
2  A person typically desires accurate results.      1   
3  A person typically desires accurate results.      0   
4  A person typically desires accurate results.      1   

                           sentence1  
0           person looking accuracy.  
1     person does not care accuracy.  
2         person double checks data.  
3  person speeds through experiment.  
4              person studying well.  


In [12]:
df_train["sentence2"] = df_train.apply(lambda row : data_preprocessing(row["sentence2_unprocessed"]), axis=1)
df_val["sentence2"] = df_val.apply(lambda row : data_preprocessing(row["sentence2_unprocessed"]), axis=1)
df_test["sentence2"] = df_test.apply(lambda row : data_preprocessing(row["sentence2_unprocessed"]), axis=1)

In [13]:
# # Save Pre Processed Text
# print(df_train.head())
# df_train.to_csv("/content/drive/MyDrive/CSCI-544/trainpy.csv", sep=',')
# df_val.to_csv("/content/drive/MyDrive/CSCI-544/valpy.csv", sep=',')
# df_test.to_csv("/content/drive/MyDrive/CSCI-544/testpy.csv", sep=',')



In [14]:
# # Pre Processed Text

# df_train = pd.read_csv('/content/drive/MyDrive/CSCI-544/train.csv', header=0,  encoding="UTF-8")
# df_val   = pd.read_csv('/content/drive/MyDrive/CSCI-544/val.csv', header=0, encoding="UTF-8")
# df_test = pd.read_csv('/content/drive/MyDrive/CSCI-544/test.csv', header=0, encoding="UTF-8")
# df_train["sentence1"] = df_train.apply(lambda row : data_preprocessing(row["sentence1"]), axis=1)
# df_val["sentence1"] = df_val.apply(lambda row : data_preprocessing(row["sentence1"]), axis=1)
# #df_val = itter_dataset(df_val, "sentence1")
# df_test["sentence1"] = df_test.apply(lambda row : data_preprocessing(row["sentence1"]), axis=1)
# df_train["sentence2"] = df_train.apply(lambda row : data_preprocessing(row["sentence2"]), axis=1)
# df_val["sentence2"] = df_val.apply(lambda row : data_preprocessing(row["sentence2"]), axis=1)
# df_test["sentence2"] = df_test.apply(lambda row : data_preprocessing(row["sentence2"]), axis=1)
# print(df_val.head())


In [15]:
# For Final training
final_train_data = pd.concat([df_train, df_val], ignore_index=True, sort=False)
print(final_train_data.head())
print(final_train_data.shape)

                               sentence1_unprocessed  \
0                             Sometimes do exercise.   
1                               Who eats junk foods.   
2                                  A person is sick.   
3                                  A person is dead.   
4  A person eats properly and do exercise regularly.   

                     sentence2_unprocessed  label  \
0  A person typically desire healthy life.      1   
1  A person typically desire healthy life.      0   
2  A person typically desire healthy life.      1   
3  A person typically desire healthy life.      0   
4  A person typically desire healthy life.      1   

                                         sentence1  \
0                           Sometimes do exercise.   
1                             Who eats junk foods.   
2                                     person sick.   
3                                     person dead.   
4  person eats properly and do exercise regularly.   

                   

In [16]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen, with_labels=True, bert_model='albert-base-v2'):

        self.data = data  
        #Initialize the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(bert_model)  

        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

        # Selecting sentence1 and sentence2 at the specified index in the data frame
        sent1 = str(self.data.loc[index, 'sentence1'])
        sent2 = str(self.data.loc[index, 'sentence2'])

        # Tokenize the pair of sentences to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(sent1, sent2, padding='max_length', truncation=True, max_length=self.maxlen, return_tensors='pt')  
        
        token_ids = encoded_pair['input_ids'].squeeze(0) 
        attn_masks = encoded_pair['attention_mask'].squeeze(0) 
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  

        if not self.with_labels: 
          return token_ids, attn_masks, token_type_ids            
        else:
          label = self.data.loc[index, 'label']
          return token_ids, attn_masks, token_type_ids, label  
            

In [17]:
class SentencePairClassifier(nn.Module):

    def __init__(self, bert_model="albert-base-v2", freeze_bert=False):
        super(SentencePairClassifier, self).__init__()
        self.bert_layer = AutoModel.from_pretrained(bert_model)
        hidden_size = 768
        self.cls_layer = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(p=0.1)

    @autocast() 
    def forward(self, input_ids, attn_masks, token_type_ids):
        cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)
        logits = self.cls_layer(self.dropout(pooler_output))
        return logits

In [25]:
def get_probs_from_logits(logits):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    return probs.detach().cpu().numpy()

def test_prediction(net, device, dataloader, with_labels=True, result_file="output.txt"):
    net.eval()
    w = open(result_file, 'w')
    probs_all = []

    test_set = CustomDataset(df_val, maxlen, bert_model)
    test_loader = DataLoader(test_set, batch_size=bs, num_workers=5)
    dataloader = test_loader

    with torch.no_grad():
      for seq, attn_masks, token_type_ids, _ in tqdm(dataloader):
          seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
          logits = net(seq, attn_masks, token_type_ids)
          probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
          probs_all += probs.tolist()
    #print(len(probs_all))


    w.writelines(str(prob)+'\n' for prob in probs_all)
    probs_test = pd.read_csv(result_file, header=None)[0] 
    threshold = 0.5   # you can adjust this threshold for your own dataset
    temp_list = []
    #print(probs_all)
    for entry in probs_all:
      if entry >= threshold:
        temp_list.append(1)
      else:
        temp_list.append(0)
    df_val["temp_val"] = temp_list
    preds_test=df_val["temp_val"].astype('uint8') # predicted labels using the above fixed threshold
    labels_test = df_val["label"].astype('uint8')

    #print(preds_test)
    #print(labels_test)

    # Evaluation Method
    metric = load_metric("glue", "mrpc")
    ans = metric._compute(predictions=preds_test, references=labels_test)
    print(ans)
    
    w.close()
    return ans['accuracy']

In [26]:
# print("Creation of the models' folder...")
# !mkdir models
# !mkdir results

In [37]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):

    besst_acc = 0
    best_ep = 0


    nb_iterations = len(train_loader)
    print_every = nb_iterations // 5  # print the training loss 5 times per epoch
   

    scaler = GradScaler()


    for ep in range(epochs):

        net.train()

        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels = seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
    
            # Enables autocasting for the forward pass (model + loss)
            with autocast():
                # Compute Loss
                logits = net(seq, attn_masks, token_type_ids)
                loss = criterion(logits.squeeze(-1), labels.float())
                loss = loss / iters_to_accumulate 

            # Backpropagating the gradients
            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:
                # Optimization step
              
                scaler.step(opti)
                scaler.update()
                lr_scheduler.step()
                opti.zero_grad()
        

        val_acc = test_prediction(net, device, val_loader)
        print("Accuracy = "+ str(val_acc))
        if val_acc > besst_acc:
            print("Best epoch " + str(ep))
            net_copy = copy.deepcopy(net)  
            besst_acc = val_acc
            best_ep = ep + 1

    # Saving the model
    path_to_model='/content/drive/MyDrive/CSCI-544/{}_lr_{}_val_loss_{}_ep_{}.pt'.format(bert_model, lr, round(besst_acc, 5), best_ep)
    torch.save(net_copy.state_dict(), path_to_model)
    #print("The model has been saved in {}".format(path_to_model))

    del loss
    torch.cuda.empty_cache()

In [38]:
bert_model = "albert-base-v2"
epochs = 7 
maxlen = 128  
bs = 16
iters_to_accumulate = 2  # the gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate. If set to "1", you get the usual batch size
lr = 2*1e-5 
freeze_bert = False 


In [39]:
# For Initial Training

# # Creating instances of training and validation set
# print("Reading training data...")
# train_set = CustomDataset(df_train, maxlen, bert_model)
# print("Reading validation data...")
# val_set = CustomDataset(df_val, maxlen, bert_model)
# # Creating instances of training and validation dataloaders
# train_loader = DataLoader(train_set, batch_size=bs, num_workers=5)
# val_loader = DataLoader(val_set, batch_size=bs, num_workers=5)


# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# net = SentencePairClassifier(bert_model, freeze_bert=freeze_bert)


# if torch.cuda.device_count() > 0:  # if multiple GPUs
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     #net = nn.DataParallel(net)

# net.to(device)

# criterion = nn.BCEWithLogitsLoss()
# opti = AdamW(net.parameters(), lr=lr, weight_decay=1e-2)
# num_warmup_steps = 0 # The number of steps for the warmup phase.
# num_training_steps = epochs * len(train_loader)  # The total number of training steps
# t_total = (len(train_loader) // iters_to_accumulate) * epochs  # Necessary to take into account Gradient accumulation
# lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)

# train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate)

In [40]:
# # Copied From https://github.com/patrickvonplaten/notebooks/blob/master/PyTorch_Reformer.ipynb
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip -q install gputil
# !pip -q install psutil
# !pip -q install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()

In [ ]:

# For Final Training

# Creating instances of training and validation set
train_set = CustomDataset(final_train_data, maxlen, bert_model)
train_loader = DataLoader(train_set, batch_size=bs, num_workers=2)

num_training_steps = epochs * len(train_loader) 
t_total = (len(train_loader) // iters_to_accumulate) * epochs 


# Creating instances of training and validation dataloaders
val_set = CustomDataset(df_val, maxlen, bert_model)
val_loader = DataLoader(val_set, batch_size=bs, num_workers=2)





device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = SentencePairClassifier(bert_model, freeze_bert=freeze_bert)

net.to(device)

criterion = nn.BCEWithLogitsLoss()
opti = AdamW(net.parameters(), lr=lr, weight_decay=2e-2)

lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_training_steps=t_total, num_warmup_steps=0 )

train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate)

100%|██████████| 440/440 [01:10<00:00,  6.28it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 66/66 [00:04<00:00, 13.80it/s]
Checking /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py for additional imports.
Found main folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/modules/datasets_modules/metrics/glue
Found specific version folder for metric https://raw.githubuserconten

{'accuracy': 0.885308056872038, 'f1': 0.8912848158131177}
Accuracy = 0.885308056872038
Best epoch 0


100%|██████████| 440/440 [01:10<00:00,  6.28it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 66/66 [00:04<00:00, 13.92it/s]
Checking /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py for additional imports.
Found main folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/modules/datasets_modules/metrics/glue
Found specific version folder for metric https://raw.githubuserconten

{'accuracy': 0.9232227488151659, 'f1': 0.9272237196765499}
Accuracy = 0.9232227488151659
Best epoch 1


 19%|█▊        | 82/440 [00:13<00:56,  6.34it/s]

In [ ]:
def find_results(with_labels=True, result_file="/content/drive/MyDrive/CSCI-544/output.txt"):
    
    net = SentencePairClassifier(bert_model)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net.load_state_dict(torch.load("/content/drive/MyDrive/CSCI-544/albert-base-v2_lr_2e-05_val_loss_0.06192_ep_7.pt"))
    net.to(device)
    net.eval()
    
    #w = open(result_file, 'w')
    probs_all = []
    rand_list = []
    for _ in range(4850):
      rand_list.append(0)
    df_test['label'] = rand_list

    test_set = CustomDataset(df_test, maxlen, bert_model)
    test_loader = DataLoader(test_set, batch_size=bs, num_workers=5)
    dataloader = test_loader

    with torch.no_grad():
      for seq, attn_masks, token_type_ids, _ in tqdm(dataloader):
          seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
          logits = net(seq, attn_masks, token_type_ids)
          probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
          probs_all += probs.tolist()
       
    #print(len(probs_all))
    #w.writelines(str(prob)+'\n' for prob in probs_all)
    #probs_test = pd.read_csv(result_file, header=None)[0]  # prediction probabilities
    threshold = 0.5   # you can adjust this threshold for your own dataset
    temp_list = []
    #print(probs_all)
    for entry in probs_all:
      if entry >= threshold:
        temp_list.append(1)
      else:
        temp_list.append(0)
    #df_val["temp_val"] = temp_list
    #preds_test=df_val["temp_val"].astype('uint8') # predicted labels using the above fixed threshold
    #labels_test = df_val["label"].astype('uint8')
    #print(preds_test)
    #print(labels_test)
    #metric = load_metric("glue", "mrpc")
    #print(metric._compute(predictions=preds_test, references=labels_test))
    #print(metric)
    results = [int(x) for x in temp_list]
    assert (len(results) == 4850)
    # write your prediction results to 'upload_predictions.txt' and upload that later
    with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
        for x in results:
            fp.write(str(x) + '\n')
    #w.close()

find_results()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 304/304 [00:19<00:00, 15.53it/s]


In [ ]:
# For testing on dev set to make sure no shuffle

# def find_results(with_labels=True, result_file="/content/drive/MyDrive/CSCI-544/output.txt"):
#     """
#     Predict the probabilities on a dataset with or without labels and print the result in a file
#     """
#     net = SentencePairClassifier(bert_model)
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     net.load_state_dict(torch.load("/content/drive/MyDrive/CSCI-544/albert-base-v2_lr_2e-05_val_loss_0.06192_ep_7.pt"))
#     net.to(device)
#     net.eval()
#     #w = open(result_file, 'w')
#     probs_all = []
#     # rand_list = []
#     # for _ in range(4850):
#     #   rand_list.append(0)
#     # df_test['label'] = rand_list

#     test_set = CustomDataset(df_val, maxlen, bert_model)
#     test_loader = DataLoader(test_set, batch_size=bs, num_workers=5)
#     dataloader = test_loader

#     with torch.no_grad():
#         if with_labels:
#             for seq, attn_masks, token_type_ids, _ in tqdm(dataloader):
#                 seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
#                 logits = net(seq, attn_masks, token_type_ids)
#                 probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
#                 probs_all += probs.tolist()
#         else:
#             for seq, attn_masks, token_type_ids in tqdm(dataloader):
#                 seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
#                 logits = net(seq, attn_masks, token_type_ids)
#                 probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
#                 probs_all += probs.tolist()
#     #print(len(probs_all))
#     #w.writelines(str(prob)+'\n' for prob in probs_all)
#     #probs_test = pd.read_csv(result_file, header=None)[0]  # prediction probabilities
#     threshold = 0.5   # you can adjust this threshold for your own dataset
#     temp_list = []
#     #print(probs_all)
#     for entry in probs_all:
#       if entry >= threshold:
#         temp_list.append(1)
#       else:
#         temp_list.append(0)
#     df_val["temp_val"] = temp_list
#     preds_test=df_val["temp_val"].astype('uint8') # predicted labels using the above fixed threshold
#     labels_test = df_val["label"].astype('uint8')
#     print(preds_test)
#     print(labels_test)
#     metric = load_metric("glue", "mrpc")
#     print(metric._compute(predictions=preds_test, references=labels_test))
#     print(metric)
#     # results = [int(x) for x in temp_list]
#     # assert (len(results) == 4850)
#     # # write your prediction results to 'upload_predictions.txt' and upload that later
#     # with open('upload_predictions2.txt', 'w', encoding = 'utf-8') as fp:
#     #     for x in results:
#     #         fp.write(str(x) + '\n')
#     #w.close()

# find_results()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 66/66 [00:04<00:00, 14.53it/s]


0       1
1       0
2       1
3       0
4       1
       ..
1050    0
1051    0
1052    1
1053    1
1054    1
Name: temp_val, Length: 1055, dtype: uint8
0       1
1       0
2       1
3       0
4       1
       ..
1050    0
1051    0
1052    1
1053    1
1054    1
Name: label, Length: 1055, dtype: uint8


Checking /root/.cache/huggingface/datasets/50d5843bbbbd80c47809bc76a5b03c0fd87d068509b0060103ae8182e4f5cfb9.ec871b06a00118091ec63eff0a641fddcb8d3c7cd52e855bbb2be28944df4b82.py for additional imports.
Found main folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/modules/datasets_modules/metrics/glue
Found specific version folder for metric https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py at /root/.cache/huggingface/modules/datasets_modules/metrics/glue/19382a5758e4e23ecb68fc5a724e719f425492cd21d2aba0db5053ec14cac0d6
Found script file from https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/glue.py to /root/.cache/huggingface/modules/datasets_modules/metrics/glue/19382a5758e4e23ecb68fc5a724e719f425492cd21d2aba0db5053ec14cac0d6/glue.py
Couldn't find dataset infos file at https://raw.githubusercontent.com/huggingface/datasets/1.0.1/metrics/glue/dataset_infos.j

{'accuracy': 0.9848341232227488, 'f1': 0.9855334538878843}
Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of translations to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
""", stored examples: 0)
